In [1]:
%%configure -f
{
    "conf": {
        "spark.pyspark.python": "python3",
        "spark.pyspark.virtualenv.enabled": "true",
        "spark.pyspark.virtualenv.type":"native",
        "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
    }
}

In [2]:
sc.install_pypi_package("pandas==1.0.5","https://pypi.org/simple")
sc.install_pypi_package("scipy==1.4.1", "https://pypi.org/simple")
sc.install_pypi_package("matplotlib==3.2.1", "https://pypi.org/simple")

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1716564517486_0005,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/af/f3/683bf2547a3eaeec15b39cef86f61e921b3b187f250fcd2b5c5fb4386369/pandas-1.0.5-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/ec/57/56b9bcc3c9c6a792fcbaf139543cee77261f3651ca9da0c93f5c1221264b/python_dateutil-2.9.0.post0-py2.py3-none-any.whl

  Using cached https://files.pythonhosted.org/packages/dd/82/c1fe128f3526b128cfd185580ba40d01371c5d299fcf7f77968e22dfcc2e/scipy-1.4.1-cp37-cp37m-manylinux1_x86_64.whl

  Using cached https://files.pythonhosted.org/packages/b2/c2/71fcf957710f3ba1f09088b35776a799ba7dd95f7c2b195ec800933b276b/matplotlib-3.2.1-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/9d/ea/6d76df31432a0e6fdf81681a895f009a4bb47b3c39036db3e1b528191d52/pyparsing-3.1.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/5c/f9/695d6bedebd747e5eb0fe8fad57b72fdf25411273a39791cde838d5a8f51/cycler-0.11.0-py3-none-any.whl
  Using cach

## This section demonstrates that my Lambda function works. 
- However, we won't use this CSV for future analysis because I don't have access to the new database right now, and our data is insufficient for running large-scale computing tasks. I bootstrapped 100,000 data points locally and uploaded them to my S3 bucket. I will use that data for analysis.

In [3]:
data = spark.read.csv('s3://reibucket/results/query_results.csv', header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
data.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- uid: string (nullable = true)
 |-- userName: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sub_category: string (nullable = true)
 |-- product: string (nullable = true)
 |-- listing_amount: string (nullable = true)
 |-- counts: string (nullable = true)
 |-- total_bid_amount: string (nullable = true)

In [5]:
data.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------------+--------+------------+--------------------+--------------+------+----------------+
|                 uid|       userName|category|sub_category|             product|listing_amount|counts|total_bid_amount|
+--------------------+---------------+--------+------------+--------------------+--------------+------+----------------+
|25b7cbf3-2040-4bb...|    WillAppleID|     Art|    Fine Art|            ndjwidnf|           4.0|     1|             4.0|
|25b7cbf3-2040-4bb...|    WillAppleID|     Art|    Fine Art|another attempt a...|          25.0|     1|            25.0|
|119f3843-43af-403...|   anamariadem7|     Art|    Fine Art|       immersive art|        1000.0|     1|          1000.0|
|0b26b2bd-3ac2-492...|   encoregoogle|     Art|    Fine Art|            ndjwidnf|           4.0|     1|             5.0|
|7feffde3-6f0e-459...|     testuser99|     Art|    Fine Art|another attempt a...|          25.0|     1|            28.0|
|119f3843-43af-403...|   anamari

## This is the csv I will actually use for the analysis: 

In [6]:
df = spark.read.csv("s3a://reibucket/Mock_data/fake_user_interaction.csv", header=True) 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- uid: string (nullable = true)
 |-- userName: string (nullable = true)
 |-- catagory: string (nullable = true)
 |-- sub_catagory: string (nullable = true)
 |-- product: string (nullable = true)
 |-- listing_amount: string (nullable = true)
 |-- counts: string (nullable = true)
 |-- total_bid_amount: string (nullable = true)

In [8]:
df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+------------------+-----------------+--------------------+--------------+------+----------------+
|                 uid|           userName|          catagory|     sub_catagory|             product|listing_amount|counts|total_bid_amount|
+--------------------+-------------------+------------------+-----------------+--------------------+--------------+------+----------------+
|7ec3c81f-4340-428...|        checkmeout_|Celebrity Signings|Movie Celebrities|Kevin Dillon Auto...|           1.0|     1|               1|
|a469955b-a20d-4cb...|        DeadStockNH|Celebrity Signings|   TV Celebrities|Bakugo Autographe...|           1.0|     4|              29|
|7c10f78f-ecfb-405...|             bankai|Celebrity Signings|   TV Celebrities|Bakugo Autographe...|           1.0|     4|              46|
|c8d09ded-5f83-40d...|Modern_Vintage_NYC |Celebrity Signings|   TV Celebrities|Bakugo Autographe...|           1.0|     3|              26|
|5dce8cf3-36fc-468..

## Data cleaning: 

In [9]:
from pyspark.sql.functions import col, isnan, count, regexp_replace, lower, explode, when,row_number

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
df.select([count(when(col(c).isNull() | isnan(c), c)).alias(c) for c in df.columns]).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+--------+--------+------------+-------+--------------+------+----------------+
|uid|userName|catagory|sub_catagory|product|listing_amount|counts|total_bid_amount|
+---+--------+--------+------------+-------+--------------+------+----------------+
|  0|       4|       4|           4|      8|             8|     8|               8|
+---+--------+--------+------------+-------+--------------+------+----------------+

In [11]:
# Drop rows where userName is empty or null
df = df.filter(df.userName.isNotNull() & (df.userName != ''))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
#fill null values in numerical columns with 0 and in string columns with 'Unknown'
df = df.na.fill({
    'listing_amount': 0, 
    'counts': 0, 
    'total_bid_amount': 0,
    'userName': 'Unknown',
    'catagory': 'Unknown', 
    'sub_catagory': 'Unknown', 
    'product': 'Unknown'
})


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
#Avoid cases where two products are actually the same, but some sellers have added non-word characters or capitalization to the title.

df = df.withColumn("product", lower(regexp_replace(col("product"), "[^a-zA-Z0-9\\s]", "")))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+------------------+-----------------+--------------------+--------------+------+----------------+
|                 uid|           userName|          catagory|     sub_catagory|             product|listing_amount|counts|total_bid_amount|
+--------------------+-------------------+------------------+-----------------+--------------------+--------------+------+----------------+
|7ec3c81f-4340-428...|        checkmeout_|Celebrity Signings|Movie Celebrities|kevin dillon auto...|           1.0|     1|               1|
|a469955b-a20d-4cb...|        DeadStockNH|Celebrity Signings|   TV Celebrities|bakugo autographe...|           1.0|     4|              29|
|7c10f78f-ecfb-405...|             bankai|Celebrity Signings|   TV Celebrities|bakugo autographe...|           1.0|     4|              46|
|c8d09ded-5f83-40d...|Modern_Vintage_NYC |Celebrity Signings|   TV Celebrities|bakugo autographe...|           1.0|     3|              26|
|5dce8cf3-36fc-468..

## Build the recommondation model: 

In [15]:
from pyspark.sql.window import Window
from pyspark.ml.feature import StringIndexer
from pyspark.ml.recommendation import ALS
import pyspark.sql.functions as F

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# Calculate the top 10 most popular items overall
top_10_popular_items = df.groupBy("product").agg(F.sum("counts").alias("total_bids")).orderBy(F.desc("total_bids")).limit(10)
top_10_popular_items.show()

# Calculate the top 10 most popular items by category and sub_category using window specification
window_spec = Window.partitionBy("catagory", "sub_catagory").orderBy(F.desc("total_bids"))

ranked_products = df.groupBy("catagory", "sub_catagory", "product") \
    .agg(F.sum("counts").alias("total_bids")) \
    .withColumn("rank", row_number().over(window_spec)) \
    .filter(col("rank") <= 10)

ranked_products.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----------+
|      product|total_bids|
+-------------+----------+
|       course|    1376.0|
|         week|    1374.0|
|     democrat|    1314.0|
|       theory|    1309.0|
|       change|    1279.0|
|international|    1278.0|
|      suggest|    1278.0|
|          tax|    1277.0|
|        think|    1274.0|
|        owner|    1274.0|
+-------------+----------+

+--------+------------+---------+----------+----+
|catagory|sub_catagory|  product|total_bids|rank|
+--------+------------+---------+----------+----+
|American|        sing| evidence|      17.0|   1|
|American|        sing|     work|      15.0|   2|
|Congress|        cold|      hot|      15.0|   1|
|Congress|    describe|     step|      18.0|   1|
|Congress|    discover|     drug|      17.0|   1|
|Democrat|    approach| cultural|       8.0|   1|
|Democrat|        face|     sell|      15.0|   1|
|Democrat|      factor|     tell|      13.0|   1|
|Democrat|      future|     note|       8.0|   1|
|Democrat|         hi

In [17]:
# Convert 'userName' and 'product' columns to numeric indices
user_indexer = StringIndexer(inputCol="userName", outputCol="userId")
product_indexer = StringIndexer(inputCol="product", outputCol="itemId")
bidding_indexer = StringIndexer(inputCol="counts", outputCol="bids")

df = user_indexer.fit(df).transform(df)
df = product_indexer.fit(df).transform(df)
df = bidding_indexer.fit(df).transform(df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
# Prepare data for ALS model
als_data = df.select(col("userId").cast("int"), col("itemId").cast("int"), col("bids").cast("int") )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:

# Train ALS model
als = ALS(
    maxIter=10,
    regParam=0.1,
    userCol="userId",
    itemCol="itemId",
    ratingCol="bids",
    coldStartStrategy="drop"
)
model = als.fit(als_data)

# Generate top 10 recommendations for each user
user_recommendations = model.recommendForAllUsers(10)
user_recommendations.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    12|[[809, 1.3478367]...|
|    65|[[807, 28.494137]...|
|    76|[[1332, 9.840526]...|
|    81|[[806, 27.868267]...|
|   122|[[732, 26.951458]...|
|   126|[[824, 20.987797]...|
|   133|[[1332, 41.71848]...|
|   140|[[305, 23.146338]...|
|   148|[[965, 25.415472]...|
|   177|[[1218, 29.734053...|
|   192|[[771, 24.102068]...|
|   209|[[615, 26.464588]...|
|   243|[[826, 5.2413816]...|
|   300|[[104, 17.73389],...|
|   333|[[897, 33.251488]...|
|   406|[[741, 27.485313]...|
|   417|[[873, 25.0877], ...|
|   444|[[1332, 39.677315...|
|   481|[[430, 22.646383]...|
|   496|[[162, 23.623482]...|
+------+--------------------+
only showing top 20 rows

## Evaluate my ALS collaborative filtering model and fine tuning the parameters

In [20]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
# set up parameters and evaluator
paramGrid = ParamGridBuilder() \
    .addGrid(als.rank, [10,20]) \
    .addGrid(als.maxIter, [10]) \
    .addGrid(als.regParam, [0.1]) \
    .build()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="bids",
    predictionCol="prediction"
)

crossval = CrossValidator(
    estimator=als,
    estimatorParamMaps=paramGrid,
    evaluator=evaluator,
    numFolds=2  # Number of folds for cross-validation
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
# Fit the ALS model to the training data
cvModel = crossval.fit(df)

# Extract the best model
bestModel = cvModel.bestModel

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
# split into train/test data
train, test = df.randomSplit([0.8, 0.2], seed=0)

# Persist data in memory to speed up I/O during training/testing
train.persist()
test.persist()

# confirm they're cached
print(train.is_cached, test.is_cached)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

True True

In [29]:
# Generate predictions on the test set
predictions = bestModel.transform(test)

# Evaluate the model
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error on test data = " + str(rmse))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root-mean-square error on test data = 0.22392845051835258

In [30]:
print("Best Model Parameters:")
print(" Rank:", bestModel.rank)
print(" MaxIter:", bestModel._java_obj.parent().getMaxIter())
print(" RegParam:", bestModel._java_obj.parent().getRegParam())


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Best Model Parameters:
 Rank: 20
 MaxIter: 10
 RegParam: 0.1

- An RMSE of 0.2239 suggests that, on average, my model's predictions are about 0.2239 units away from the actual data points in terms of the bids rating scale. However, this is a fake dataset, so I will need to fine tuning it more after words. 

## Show the final recommendations with original user names and product names

In [33]:
# Generate top 10 recommendations for each user
user_recommendations = bestModel.recommendForAllUsers(10)
user_recommendations.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    12|[[462, 2.7344208]...|
|    65|[[833, 24.765823]...|
|    76|[[1332, 11.110581...|
|    81|[[624, 22.323032]...|
|   122|[[128, 24.945017]...|
|   126|[[1125, 23.439484...|
|   133|[[705, 18.822424]...|
|   140|[[399, 19.345722]...|
|   148|[[925, 19.97949],...|
|   177|[[957, 20.058804]...|
|   192|[[653, 17.96259],...|
|   209|[[519, 19.876812]...|
|   243|[[1332, 4.072618]...|
|   300|[[768, 16.290277]...|
|   333|[[795, 21.36091],...|
|   406|[[663, 17.360752]...|
|   417|[[850, 17.768967]...|
|   444|[[354, 18.322456]...|
|   481|[[965, 17.488764]...|
|   496|[[301, 19.924335]...|
+------+--------------------+
only showing top 20 rows

In [36]:
# Create a DataFrame for userId to userName mapping
user_id_mapping = df.select("userId", "userName").distinct()

# Create a DataFrame for itemId to product mapping
item_id_mapping = df.select("itemId", "product").distinct()

# Explode the recommendations
exploded_recommendations = user_recommendations.withColumn("recommendation", explode("recommendations"))

# Select userId and the recommended itemId and rating
exploded_recommendations = exploded_recommendations.select("userId", col("recommendation.itemId").alias("itemId"), col("recommendation.rating").alias("predicted_bids"))

# Join with user_id_mapping to get userName
recommendations_with_user = exploded_recommendations.join(user_id_mapping, on="userId")

# Join with item_id_mapping to get product
recommendations_with_user_product = recommendations_with_user.join(item_id_mapping, on="itemId")

# Select relevant columns
recommendations_final = recommendations_with_user_product.select("userName", "product", "predicted_bids")

# Show the final recommendations with original user names and product names
recommendations_final.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-------------+--------------+
| userName|      product|predicted_bids|
+---------+-------------+--------------+
|      geo|         girl|     2.7344208|
|      geo|        focus|     2.6825073|
|      geo|        cover|     2.6204603|
|      geo|        visit|     2.6065812|
|      geo|       mother|     2.5866444|
|      geo|     pressure|     2.5358653|
|      geo|        happy|     2.5160544|
|      geo|           on|     2.4734707|
|      geo|      foreign|     2.3637054|
|      geo|       others|     2.3560643|
|uwilliams|      science|     24.765823|
|uwilliams|        crime|     24.364996|
|uwilliams|          but|     23.212934|
|uwilliams|          add|     22.127392|
|uwilliams|         game|     21.541166|
|uwilliams|         also|     21.478506|
|uwilliams|          you|     21.406864|
|uwilliams|         loss|     21.377327|
|uwilliams|international|     20.521315|
|uwilliams|      program|     20.402838|
+---------+-------------+--------------+
only showing top

In [41]:
# Calculate the sum of counts for each user
user_counts = df.groupBy("userName").agg(F.sum("counts").alias("total_counts_users"))
# Join the sum of counts back to the original DataFrame
df_with_counts = df.join(user_counts, "userName", "inner")

# Show the resulting DataFrame
df_with_counts.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------------------+------------------+-----------------+--------------------+--------------+------+----------------+------+------+----+------------------+
|           userName|                 uid|          catagory|     sub_catagory|             product|listing_amount|counts|total_bid_amount|userId|itemId|bids|total_counts_users|
+-------------------+--------------------+------------------+-----------------+--------------------+--------------+------+----------------+------+------+----+------------------+
|        checkmeout_|7ec3c81f-4340-428...|Celebrity Signings|Movie Celebrities|kevin dillon auto...|           1.0|     1|               1| 242.0|1505.0| 0.0|              13.0|
|        DeadStockNH|a469955b-a20d-4cb...|Celebrity Signings|   TV Celebrities|bakugo autographe...|           1.0|     4|              29|  23.0|1188.0| 3.0|              66.0|
|             bankai|7c10f78f-ecfb-405...|Celebrity Signings|   TV Celebrities|bakugo autographe...|          

In [42]:
df_with_counts.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- userName: string (nullable = false)
 |-- uid: string (nullable = true)
 |-- catagory: string (nullable = false)
 |-- sub_catagory: string (nullable = false)
 |-- product: string (nullable = false)
 |-- listing_amount: string (nullable = false)
 |-- counts: string (nullable = false)
 |-- total_bid_amount: string (nullable = false)
 |-- userId: double (nullable = false)
 |-- itemId: double (nullable = false)
 |-- bids: double (nullable = false)
 |-- total_counts_users: double (nullable = true)

## Combining popularity model + collaborative filtering 
- Here we arbitrarily choose ("total_counts_users") == 0 to avoid the code start problem but this parameter can be further adjusted. 

In [43]:
from pyspark.sql.functions import col, broadcast

def recommend_items(user_data, top_10_popular, top_10_category_subcategory, recommendations_final):
    # Filter users with more than zero counts and known categories
    condition_users = user_data.filter(
        (col("total_counts_users") == 0) &
        ((col("catagory") != 'unknown') | (col("sub_catagory") != 'unknown'))
    )

    # Join with top categories/subcategories and rename the 'product' column
    recommendations_category = condition_users.join(
        broadcast(top_10_category_subcategory),
        (condition_users["catagory"] == top_10_category_subcategory["catagory"]) &
        (condition_users["sub_catagory"] == top_10_category_subcategory["sub_catagory"]),
        "left"
    ).select(condition_users["userName"], top_10_category_subcategory["product"].alias("recommended_product"), "total_bids")

    # Users with zero or unknown category/subcategory get popular items
    default_users = user_data.filter(
        (col("total_counts_users") == 0) |
        ((col("catagory") == 'unknown') & (col("sub_catagory") == 'unknown'))
    )
    recommendations_default = default_users.crossJoin(broadcast(top_10_popular)).select(
        "userName", top_10_popular["product"].alias("popular_product"), "total_bids"
    )

    # Users needing predictions from the final model
    model_users = user_data.filter(col("total_counts_users") > 0)
    recommendations_model = model_users.join(
        recommendations_final, "userName"
    ).select("userName", recommendations_final["product"].alias("model_product"), "predicted_bids")

    # Combine all recommendations
    # Here you may need to standardize column names before union, or handle it differently based on your needs.
    final_recommendations = recommendations_category.withColumnRenamed("recommended_product", "product") \
        .union(recommendations_default.withColumnRenamed("popular_product", "product")) \
        .union(recommendations_model.withColumnRenamed("model_product", "product"))

    return final_recommendations


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
# Apply the function (example usage)
final_recommendations_df = recommend_items(df_with_counts, top_10_popular_items, ranked_products, recommendations_final)
final_recommendations_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------+------------------+
|userName| product|        total_bids|
+--------+--------+------------------+
| vmiller|      in|19.979490280151367|
| vmiller| morning|19.979490280151367|
| vmiller|    save|19.979490280151367|
| vmiller|  relate|19.979490280151367|
| vmiller|    poor|19.979490280151367|
| vmiller| college|19.979490280151367|
| vmiller|  relate|19.979490280151367|
| vmiller|   clear|19.979490280151367|
| vmiller|   every|19.979490280151367|
| vmiller|building|19.979490280151367|
| vmiller|  appear|19.979490280151367|
| vmiller|    fall|19.979490280151367|
| vmiller|    none|19.979490280151367|
| vmiller|     kid|19.979490280151367|
| vmiller|      in|18.377710342407227|
| vmiller| morning|18.377710342407227|
| vmiller|    save|18.377710342407227|
| vmiller|  relate|18.377710342407227|
| vmiller|    poor|18.377710342407227|
| vmiller| college|18.377710342407227|
+--------+--------+------------------+
only showing top 20 rows

## Sample recommendations for a user 

In [45]:
def get_top_recommendations_for_user(recommendations_df, user_name, top_n=10):
    """
    Fetch top N recommendations for a specific user from the DataFrame of recommendations.

    :param recommendations_df: DataFrame containing recommendations for all users.
    :param user_name: String, the userName for whom recommendations are to be fetched.
    :param top_n: Integer, number of top recommendations to fetch.
    :return: DataFrame containing top N recommendations specifically for the provided userName.
    """
    # Filter the DataFrame for the specific user
    user_recommendations = recommendations_df.filter(col("userName") == user_name)
    
    # You might already have limited recommendations or need to order and limit them based on some criteria
    # For example, if you have a score or bids column that dictates the strength of the recommendation:
    # user_recommendations = user_recommendations.orderBy(col("some_score_column").desc()).limit(top_n)

    return user_recommendations.limit(top_n)

# Example usage:
specific_user_recommendations = get_top_recommendations_for_user(final_recommendations_df, "steve", top_n=10)

# Show the recommendations for "Steve"
specific_user_recommendations.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+------------------+
|userName|             product|        total_bids|
+--------+--------------------+------------------+
|   steve|kevin dillon auto...| 8.160733222961426|
|   steve|kevin dillon auto...| 7.054625034332275|
|   steve|kevin dillon auto...|6.7168354988098145|
|   steve|kevin dillon auto...| 6.084837436676025|
|   steve|kevin dillon auto...| 5.918649673461914|
|   steve|kevin dillon auto...| 5.887824058532715|
|   steve|kevin dillon auto...| 5.811747074127197|
|   steve|kevin dillon auto...| 5.776914596557617|
|   steve|kevin dillon auto...| 5.713441848754883|
|   steve|kevin dillon auto...|  5.63450288772583|
+--------+--------------------+------------------+

- the result doesn't looks realy idea because it's purely bootstrapped

## Convert the final prediction into a json file and save it into my S3 for later access 

In [46]:
from pyspark.sql.functions import collect_list, struct

# Assuming 'final_recommendations_df' is your DataFrame
# Group by 'userName' and collect products into a list
user_recommendations = final_recommendations_df.groupBy("userName").agg(
    collect_list(struct("product", "total_bids")).alias("recommendations")
)

# Show sample data
user_recommendations.show(truncate=False)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [47]:
import json

# Convert DataFrame to JSON
json_rdd = user_recommendations.toJSON()
#json_rdd.saveAsTextFile("path/to/output/json")  # Intermediate save, not to S3 yet

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
# Define your S3 bucket and path
output_path = "s3://reibucket/recommendations"

user_recommendations.write.mode("overwrite").json(output_path)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…